## Word level Seq to Seq
Note that this encoder decoder model is a word level encoder decoder model

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

C:\Users\Rajdeep\AppData\Local\conda\conda\envs\tfdeeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 64 # declaring the batch size

In [3]:
epochs = 100 # number of epochs to train

In [4]:
latent_dim = 256

In [5]:
num_samples = 1000

In [6]:
import os, sys

In [67]:
data_path = os.path.join(os.path.dirname(os.getcwd()), "datasets", "fra-eng", "fra.txt") # path to the corpus file
# note that it is super important to know the format of the corpus file
# in this case the format is such
# <eng-text><tab><french-translation of the same eng text>
# each new training example is in a new line

In [137]:
input_texts = []
target_texts = []
# input_characters = set()
# target_characters = set()
input_words = set()
target_words = set()

In [138]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n") # spliting training examples

# remember we are trying to get only `sample` number of lines and not any more for training 
for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    
    target_text = '\t' + target_text + "\n"
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # including chars in input and target characters
#     for char in input_text:
#         if char not in input_characters:
#             input_characters.add(char)
    
#     for char in target_text:
#         if char not in target_characters:
#             target_characters.add(char)
    for word in input_text.split(" "):
        if word not in input_words:
            input_words.add(word)
    #note that the target words need to contain the tab and newline as separate words
    for word in target_text.split(" "):
        word = word.replace("\t", "")
        word = word.replace("\t", "")
        if word not in target_words:    
            target_words.add(word)
    
target_words.add("\n")
target_words.add("\t")

In [120]:
# here input_characters is the total number of input characters
# input_characters = sorted(list(input_characters))
# target_characters = sorted(list(target_characters))
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))

# num_encoder_tokens = len(input_characters)
# num_decoder_tokens = len(target_characters)
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)


# max_encoder_seq_length = max([len(txt) for txt in input_texts])
# max_decoder_seq_length = max([len(txt) for txt in target_texts])
max_encoder_seq_length = max([len(txt.split(" ")) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split(" ")) for txt in target_texts])

print("Number of samples: " + str(len(input_texts))) # total number of samples that we are training
print("Number of unique input tokens: " + str(num_encoder_tokens))
print("Number of unique output tokens: " + str(num_decoder_tokens))
print("Max sequence length for inputs: " + str(max_encoder_seq_length))
print("Max sequence length for outputs: " + str(max_decoder_seq_length))


Number of samples: 1000
Number of unique input tokens: 467
Number of unique output tokens: 938
Max sequence length for inputs: 3
Max sequence length for outputs: 7


In [121]:
# note that here we are only taking the first 1000 training examples

In [139]:
# input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
input_token_index = dict([(word, i) for i, word in enumerate(input_words)])
# recall that input_characters is a list of all the unique input characters that we have in the 1000 training examples
# that we have read so far

# what enumerate does is that it assigns a counter to the unique characters, so basically if we get the following list
# a = ["ola", "mundo", "keras"], we would get back from enumerate 1, "ola"; 2, "mundo"; 3, "keras"

# target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
target_token_index = dict([(word, i) for i, word in enumerate(target_words)])

In [140]:
# here is the important part as we are defining what goes into the input data
# max_encoder seq length is the maximum number of characters in one training example in the entire corpus
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

# the number of input_texts and the number of target_texts will be the same
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

# here we just created the vectors that we are going to use as inputs for the encoder and inputs and targets for the 
# decoder

In [143]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    input_text_words = input_text.split(" ") # the following 2 lines to be included for word level enc-dec
    target_text_words = target_text.split(" ")
    for t, word in enumerate(input_text_words):
        # i is the first index as it is the ith training example 
        # t is the second index as the t-th character in ith training example
        encoder_input_data[i, t, input_token_index[word]] = 1
    for t, word in enumerate(target_text_words):
        # here a tb splitting must be done in order to treat the tab as a separate word
        if "\t" in word:
            tab_split_words = word.split("\t")
            decoder_input_data[i, t, target_token_index["\t"]] = 1
            decoder_input_data[i, t, target_token_index[tab_split_words[1]]] = 1
        else:
            decoder_input_data[i, t, target_token_index[word]] = 1
        
        # Here is an important thing to understand
        ###############---------------------------IMPORTANT------------------------#####################
        # The decoder_target_data will be ahead of the decoder_input_data by one time step and this is an important factor
        # The decoder_target_data will obviously not contain the first character
        if t > 0:
            if "\t" in word:
                tab_split_words = word.split("\t")
                decoder_target_data[i, t-1, target_token_index["\t"]] = 1
                decoder_target_data[i, t-1, target_token_index[tab_split_words[1]]] = 1
            else:
                decoder_target_data[i, t - 1, target_token_index[word]] = 1
        

In [144]:
encoder_inputs = Input(shape=(None, num_encoder_tokens)) # num_encoder_tokens is essentially the vocab size

In [145]:
encoder = LSTM(latent_dim, return_state=True)
# Each unit or cell of LSTM within the layer has an internal cell state referred to as c and a hidden state referred
# to as h. 
# If we need to access the internal state c of the last time step we need to use return_state = True

In [146]:
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# here the encoder_outputs would be hidden state at last time step, state_h would be the hidden state of the last time step
# again and state_c is the internal state of the last time step

In [147]:
encoder_states = [state_h, state_c] # we will only keep the states and discard the output

In [148]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

In [149]:
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# return sequences will return the hidden states for each and every time step

In [150]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# note here we are passing the encoder_states as the initial_state of the decoder lstm
# decoder_outputs will actially contain the hidden states of all the time steps as an array

In [151]:
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# Dense is an ordinary dense neural network. 

In [152]:
decoder_outputs = decoder_dense(decoder_outputs)

In [153]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [154]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [155]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# passing all the data here

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 9s 11ms/step - loss: 1.5936 - val_loss: 1.6494
Epoch 2/100
800/800 [==============================] - 6s 7ms/step - loss: 1.2826 - val_loss: 1.6666
Epoch 3/100
800/800 [==============================] - 5s 7ms/step - loss: 1.1909 - val_loss: 1.7169
Epoch 4/100
800/800 [==============================] - 5s 7ms/step - loss: 1.1537 - val_loss: 1.7851
Epoch 5/100
800/800 [==============================] - 5s 7ms/step - loss: 1.1231 - val_loss: 1.7901
Epoch 6/100
800/800 [==============================] - 6s 7ms/step - loss: 1.0936 - val_loss: 1.8370
Epoch 7/100
800/800 [==============================] - 6s 7ms/step - loss: 1.0603 - val_loss: 1.9167
Epoch 8/100
800/800 [==============================] - 6s 7ms/step - loss: 1.0250 - val_loss: 1.9317
Epoch 9/100
800/800 [==============================] - 6s 7ms/step - loss: 0.9932 - val_loss: 2.0031
Epoch 10/100
800/800 [======================

Epoch 81/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0871 - val_loss: 2.5158
Epoch 82/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0845 - val_loss: 2.5284
Epoch 83/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0834 - val_loss: 2.5136
Epoch 84/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0813 - val_loss: 2.4918
Epoch 85/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0825 - val_loss: 2.5324
Epoch 86/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0806 - val_loss: 2.4848
Epoch 87/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0793 - val_loss: 2.5096
Epoch 88/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0788 - val_loss: 2.5399
Epoch 89/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0774 - val_loss: 2.5293
Epoch 90/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0771

In [156]:
model.save('seq2seqKeras1000SamplesWordLevel.h5')

C:\Users\Rajdeep\AppData\Local\conda\conda\envs\tfdeeplearning\lib\site-packages\keras\engine\topology.py:2379: UserWarning: Layer lstm_10 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_9/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_9/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [157]:
# Inference Model (Sampling sequences)
# For sampling the sequences here is the strategy to be followed

# 1. Encode the input and retrieve the initial encoder state
# 2. Run one step of the decoder with its initial encoder state and a "start of sequence" token as target
# 3. Output will be the next target token and current states

In [158]:
# In the original keras example, they are using the training examples themselves to predict the french statements
# However we would like to use predict new statements
# For this we would have to change a few things in order to achieve what we need to achieve

In [171]:
# preparing the test data
test_data_path = os.path.join(os.path.dirname(os.getcwd()), "datasets", "fra-eng", "fra-test.txt") # path to the test 
# corpus file
test_input_texts = []
with open(test_data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n") # spliting training examples

# remember we are trying to get only `sample` number of lines and not any more for training 
for line in lines:
    input_text = line
    
#     target_text = '\t' + target_text + "\n"
    
    test_input_texts.append(input_text)
#     target_texts.append(target_text)
    
encoder_test_input_data = np.zeros((len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for i, (test_input_text) in enumerate(test_input_texts):
    print("i now:" + str(i))
    test_input_text = test_input_text.split(" ")
    for t, word in enumerate(test_input_text):
        # i is the first index as it is the ith training example 
        # t is the second index as the t-th character in ith training example
#         print("t, char now: " + str(t) + ", " + str(char))
        encoder_test_input_data[i, t, input_token_index[word]] = 1

i now:0
i now:1


In [172]:
encoder_model = Model(encoder_inputs, encoder_states)

In [173]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [174]:
# reverse_input_char_index = dict(
#     (i, char) for char, i in input_token_index.items())
# reverse_target_char_index = dict(
#     (i, char) for char, i in target_token_index.items())
reverse_input_word_index = dict(
    (i, word) for word, i in input_token_index.items())
reverse_target_word_index = dict(
    (i, word) for word, i in target_token_index.items())

In [175]:
def decode_sequence(input_seq):
    """
    This function is for decoding a sequence of english letters
    """
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq) # this is actually encoder_input data

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    # target_sequence is simply a one hot vector of size vocab size

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # predicting one character at a time
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += " " + sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [176]:
for seq_index in range(2):
    # this trick below is just for converting input_seq to a list
    input_seq = encoder_test_input_data[seq_index: seq_index + 1] # we will need to form a new encoder_input_data
    decoded_sentence = decode_sequence(input_seq=input_seq)
    print('-')
    print("Input sentence: ", test_input_texts[seq_index])
    print("Decoded sentence: ", decoded_sentence)
    

-
Input sentence:  No Cheers!
Decoded sentence:   n'en !

-
Input sentence:  Jump. on.
Decoded sentence:   à rouler
